# Useful Pandas Techniques
Excellent source for core Pandas code - https://www.analyticsvidhya.com/blog/2016/01/12-pandas-techniques-python-data-manipulation/
## Boolean Indexing
Sometimes you will want to cutdown your raw data based on certain criteria in order to show you just the sub-sample that you are after. Below, you can see our data file has information on whether or not individuals are graduates, their gender and if they have taken out a loan.

In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as plt

In [2]:
df = pd.read_csv(r"C:\Users\AllenM\Desktop\Data Science\Basic Tutorial - Vidhya\Boolean Indexing.csv")
df.head(5)

,Loan_ID,Gender,Education,Loan_Status
0,ST111,M,Not Graduate,Y
1,ST112,M,Graduate,Y
2,ST113,F,Graduate,Y
3,ST114,M,Not Graduate,Y
4,ST115,F,Graduate,Y


We want to separate out the respondents who are female non-graduates with a loan, so we do the following.

In [3]:
df_sub = df.loc[(df['Gender']=="F") & (df['Education']=="Not Graduate") & (df['Loan_Status']=="Y")]
df_sub.head(5)

,Loan_ID,Gender,Education,Loan_Status
10,ST121,F,Not Graduate,Y
13,ST124,F,Not Graduate,Y


## Apply Function
*Apply* goes through each row or column of a dataframe and returns a value based on an input function.

In the below example, we will remove all null values from the data table

In [4]:
df_apply = pd.read_csv(r"C:\Users\AllenM\Desktop\Data Science\Basic Tutorial - Vidhya\Apply Function.csv")
df_apply.head(5)

,Loan_ID,Gender,Education,Loan_Status
0,ST111,M,NaN,Y
1,ST112,M,Graduate,Y
2,ST113,F,Graduate,Y
3,ST114,NaN,Not Graduate,NaN
4,ST115,NaN,Graduate,Y


In [5]:
# create function to apply
def num_missing(x):
    return sum(x.isnull())

# apply to cols
print("Number of nulls per column:")
print(df_apply.apply(num_missing, axis=0)) # axis=0 tells it to run on cols

# apply to rows
print("Number of nulls per row:")
print(df_apply.apply(num_missing, axis=1)) # axis=1 tells it to run on rows

Number of nulls per column:
Loan_ID        0
Gender         7
Education      5
Loan_Status    4
dtype: int64
Number of nulls per row:
0     1
1     0
2     0
3     2
4     1
5     2
6     2
7     0
8     0
9     0
10    1
11    0
12    0
13    0
14    2
15    3
16    1
17    0
18    1
19    0
dtype: int64


## Impute Missing Values
The most basic solution for handling missing values is imputing data into them, there are many ways to determine what to impute in their place ranging from simple 0s up to complex statistical methods for calculating the best value based on other parameters.

In this case we will simply replace them with the mode of the same variable.

In [6]:
from scipy.stats import mode

df_fill = pd.read_csv(r"C:\Users\AllenM\Desktop\Data Science\Basic Tutorial - Vidhya\Impute NAs.csv")
df_fill.head(5)

,Loan_ID,Gender,Education,Loan_Status
0,ST111,M,NaN,Y
1,ST112,M,Graduate,Y
2,ST113,F,Graduate,Y
3,ST114,NaN,Not Graduate,NaN
4,ST115,NaN,Graduate,Y


In [7]:
df_fill['Gender'].fillna(mode(df_fill['Gender'])[0], inplace=True)
df_fill['Education'].fillna(mode(df_fill['Education'])[0], inplace=True)
df_fill['Loan_Status'].fillna(mode(df_fill['Loan_Status'])[0], inplace=True)

print("Column nulls:")
print(df_fill.apply(num_missing, axis=0))
print("Row nulls:")
print(df_fill.apply(num_missing, axis=1))

df_fill.head(5)

C:\Program Files\Anaconda3\lib\site-packages\scipy\stats\stats.py:250: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


TypeError: '>' not supported between instances of 'str' and 'float'

## Pivot Tables
You can create Excel style pivot tables with Python, here we will present loan amount as the value with gender, education and loan status as indexes. There are null values so we will impute by calculating the mean from the 3 index columns.

In [16]:
df_piv = pd.read_csv(r"C:\Users\AllenM\Desktop\Data Science\Basic Tutorial - Vidhya\Pivot Table.csv")
df_piv.head(5)

,Loan_ID,Gender,Education,Loan_Status,Loan_Amount
0,ST111,M,Graduate,Y,10.00
1,ST112,M,Graduate,Y,30.54
2,ST113,F,Graduate,Y,21.02
3,ST114,F,Not Graduate,N,11.00
4,ST115,F,Graduate,Y,91.40


In [17]:
df_imp = df_piv.pivot_table(values=["Loan_Amount"], index=["Gender", "Education", "Loan_Status"], aggfunc=np.mean)
print(df_imp)

                                 Loan_Amount
Gender Education    Loan_Status             
F      Graduate     N                  6.290
                    Y                 25.434
       Not Graduate N                 10.085
                    Y                 13.480
M      Graduate     N                  2.700
                    Y                 16.780
       Not Graduate N                  6.895


## Crosstab
You can create simple variable cross-tabs in Python as well using a single line of code, these can be useful for all sorts of analysis from initial investigation to presentation of final data.

In [20]:
df_cross = pd.read_csv(r"C:\Users\AllenM\Desktop\Data Science\Basic Tutorial - Vidhya\Crosstab.csv")
df_cross.head(5)

,Loan_ID,Gender,Education,Loan_Status,Loan_Amount,Credit_Status
0,ST111,M,Graduate,Y,10.00,1
1,ST112,M,Graduate,Y,30.54,1
2,ST113,F,Graduate,Y,21.02,1
3,ST114,F,Not Graduate,N,11.00,0
4,ST115,F,Graduate,Y,91.40,0


In [22]:
pd.crosstab(df_cross["Loan_Status"], df_cross["Credit_Status"], margins=True)

Credit_Status,0,1,All
Loan_Status,,,
N,2,7,9
Y,7,4,11
All,9,11,20


This is a useful look at 2 variables, however converting the values to percentages can give us a more intuitive feel for the distribution of values in this crosstab. This can be done by applying a conversion function to the data.

In [32]:
def perc_conv(ser):
    return ser/float(ser[-1])

pd.crosstab(df_cross["Loan_Status"], df_cross["Credit_Status"], margins=True).apply(perc_conv, axis=1)

Credit_Status,0,1,All
Loan_Status,,,
N,0.222222,0.777778,1.0
Y,0.636364,0.363636,1.0
All,0.450000,0.550000,1.0
